In [12]:
! pip install ipywidgets
! jupyter nbextension enable --py widgetsnbextension

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: kernel kernelspec migrate run troubleshoot

Jupyter command `jupyter-nbextension` not found.


In [13]:
from pyspark.sql import SparkSession
from ydata_profiling import ProfileReport

In [14]:
spark = SparkSession \
    .builder \
    .appName("EDA") \
    .getOrCreate()

In [15]:
df = spark.read.option("header", True).option("inferSchema", True).csv("original_dataset/Amazon Sale Report.csv").drop('index')

In [16]:
df.show()

+-------------------+--------+--------------------+----------+-------------+------------------+--------+----+--------------+---+--------+------+-----------+--------------+----------------+------------+-----+------------+----+--------+
|           Order ID|    Date|              Status|Fulfilment|Sales Channel|ship-service-level|Category|Size|Courier Status|Qty|currency|Amount|  ship-city|    ship-state|ship-postal-code|ship-country|  B2B|fulfilled-by| New|PendingS|
+-------------------+--------+--------------------+----------+-------------+------------------+--------+----+--------------+---+--------+------+-----------+--------------+----------------+------------+-----+------------+----+--------+
|405-8078784-5731545|04-30-22|           Cancelled|  Merchant|    Amazon.in|          Standard| T-shirt|   S|    On the Way|  0|     INR|647.62|     MUMBAI|   MAHARASHTRA|          400081|          IN|false|   Easy Ship|NULL|    NULL|
|171-9198151-1101146|04-30-22|Shipped - Deliver...|  Merchan

In [17]:
df.printSchema()

root
 |-- Order ID: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Fulfilment: string (nullable = true)
 |-- Sales Channel: string (nullable = true)
 |-- ship-service-level: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Size: string (nullable = true)
 |-- Courier Status: string (nullable = true)
 |-- Qty: integer (nullable = true)
 |-- currency: string (nullable = true)
 |-- Amount: double (nullable = true)
 |-- ship-city: string (nullable = true)
 |-- ship-state: string (nullable = true)
 |-- ship-postal-code: integer (nullable = true)
 |-- ship-country: string (nullable = true)
 |-- B2B: boolean (nullable = true)
 |-- fulfilled-by: string (nullable = true)
 |-- New: string (nullable = true)
 |-- PendingS: string (nullable = true)



In [18]:
from pyspark.sql.functions import col, to_date
df = df.withColumn("Date", to_date(col("Date"), "yyyy-MM-dd"))

In [19]:
pandas_df = df.toPandas()
report = ProfileReport(pandas_df, title="Profiling Pandas DataFrame")
report.to_file('profile.html')
report.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/home/ahmad/repo/Data-Quality-with-Nessie/env/lib/python3.10/site-packages/ydata_profiling/model/pandas/summary_pandas.py:39: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  series = series.fillna(np.nan)
/home/ahmad/repo/Data-Quality-with-Nessie/env/lib/python3.10/site-packages/ydata_profiling/model/pandas/summary_pandas.py:39: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  series = series.fillna(np.nan)
/home/ahmad/repo/Data-Quality-with-Nessie/env/lib/python3.10/site-packages/ydata_profiling/model/pandas/summary_pandas.py:39: FutureWarning: Downcasting o

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [20]:
spark.stop()

# Insights

## Correlation Heat Map
![correlation](./assets/notebook/corr_map.png)

From the map above, it seems that:
- Amount and Sales Channel has high correlation =<0.75
- Category and size seems to have medium correlation 0.25<corr<0.75
- Category and amount seems to have medium correlation 0.25<corr<0.75

